# ONTOP-Dremio Playground


**NOTE**: this notebook is written in JavaScript, so you need Jupyter + a JavaScript interpreter to run it. You may use the Deno interpreter, which is powerful and comes with a prebuild support for notebooks
```bash
curl -fsSL https://deno.land/install.sh | sh  # Install Deno
"$HOME/.deno/bin/deno" jupyter --install      # Add Deno to Jupyter
```
now just restart VSCode/Jupyter and you should see Deno amongst the kernels

In [1]:
const hello = "hello world!"
console.log(hello)

hello world!


In [3]:
// start docker compose if not already
const cmd = new Deno.Command("docker", { args: ["compose", "up", "-d"], stdout: "piped", stderr: "piped",}); 
const { success, stdout, stderr } = await cmd.output();
if (success) {
  console.log("Docker Compose started successfully!");
  console.log(new TextDecoder().decode(stdout));
} else {
  console.error(new TextDecoder().decode(stderr));
}

Docker Compose started successfully!



## Setup

In [1]:
import * as u from '../utils/index.ts'

// Connect to postgres
const pg1 = await u.pgCreateClient(5531)
const pg2 = await u.pgCreateClient(5532)
const pg3 = await u.pgCreateClient(5533)

// Connect to dremio and add postgres sources if not already there
const drm = await u.drmCreateClient()
try {
  await Promise.all([
    u.drmSetupPostgres(drm,"postgres1", "postgres1"),
    u.drmSetupPostgres(drm, "postgres2", "postgres2"),
    u.drmSetupPostgres(drm, "postgres3", "postgres3"),
  ]);
}catch(e){
  if(e.httpStatus === 409) console.log("Postgres sources already there - status 409")
  else throw e
}


// Check if everything is up
console.log('')
await u.pgLogTables(pg1)
console.log(await u.drmApi(drm, 'GET', '/catalog'))

User alredy there, no need to create a new one
Auth token: ouarat9p9bge01fvknmr4u3s1i - expires at: 17:10:14 (30hours)
Postgres sources already there - status 409

Tables: [ { table_name: "patients" }, { table_name: "blood_tests" } ]
{
  data: [
    {
      id: "b98fb98d-325b-4ced-81a1-746c89f51191",
      path: [ "@dremioUser" ],
      tag: "0f0G7zHl6lk=",
      type: "CONTAINER",
      containerType: "HOME",
      createdAt: "2024-09-11T13:56:49.132Z"
    },
    {
      id: "c65e9f19-d8af-4a2c-ad3e-87164200707e",
      path: [ "postgres1" ],
      tag: "6QQBPYG6DLo=",
      type: "CONTAINER",
      containerType: "SOURCE",
      createdAt: "2024-09-11T14:07:50.368Z"
    },
    {
      id: "952856e4-5ec2-4ead-8549-14059216eb7b",
      path: [ "postgres2" ],
      tag: "p0GWIRcPKDM=",
      type: "CONTAINER",
      containerType: "SOURCE",
      createdAt: "2024-09-11T14:08:06.867Z"
    },
    {
      id: "ee9cf94e-7850-40d3-9d1c-377711d061ca",
      path: [ "postgres3" ],
      tag: "

In [5]:
u.settings.debug = false

await u.pgClear(pg1, pg2, pg3)
await u.pgLoadSQLData(pg1, './data/postgres/hospital.sql')
await u.pgLoadSQLData(pg2, './data/postgres/hospital.sql')

console.log(await u.ontopRunQuery({}, `
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX : <http://example.org/> 

    SELECT * WHERE {
      ?sub a :Patient.
    } 
  
    `))
    

sub
http://example.org/patients2/4
http://example.org/patients2/3
http://example.org/patients2/7
http://example.org/patients2/8
http://example.org/patients2/1
http://example.org/patients2/5
http://example.org/patients2/10
http://example.org/patients2/6
http://example.org/patients2/2
http://example.org/patients2/9

